In [3]:
import sparknlp
# spark = sparknlp.start() 
spark = sparknlp.start(apple_silicon=True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import *
import pyspark
print(f"Spark NLP version: {sparknlp.version()}")
print(f"Spark version: {pyspark.__version__}")



Spark NLP version: 6.2.3
Spark version: 3.3.1


In [4]:
MODEL_NAME = "sentimentdl_use_twitter"

documentAssembler = DocumentAssembler()\
    .setInputCol("reviewText")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler,
        use,
        sentimentdl
        ])

print("Pipeline defined successfully!")

tfhub_use download started this may take some time.
25/12/24 16:12:28 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
25/12/24 16:12:32 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
Approximate size to download 11.4 MB
[OK!]
Pipeline defined successfully!


In [5]:
print("\nFitting pipeline structure (no training involved)...")
empty_df = spark.createDataFrame([[""]], ["reviewText"])

pipelineModel = nlpPipeline.fit(empty_df)
print("Pipeline ready for inference!")


Fitting pipeline structure (no training involved)...
Pipeline ready for inference!


In [14]:
# Wrap your trained model
light_pipeline = LightPipeline(pipelineModel)

# Instant inference on a simple string (no Spark overhead)
result = light_pipeline.annotate("Instant inference on a simple string (no Spark overhead)")

In [17]:
if "sentiment" in result:
    print(result["sentiment"][1])
else:
    print("No sentiment found")

IndexError: list index out of range